In [3]:
import pandas as pd
import geopandas as gpd
from geopandas import GeoDataFrame
import skmob

## Reading the DataFrame

### Data original

In [ ]:
path = './DataGPS/Januari2022.csv'
df = pd.read_csv(path)
df = df.drop({'Unnamed: 0'},axis=1)
df.head()

/tmp/ipykernel_3828/2349169461.py:2: DtypeWarning: Columns (2,3,4) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(path)


,maid,latitude,longitude,timestamp
0,8f60c037-8ff2-4ccd-bd6d-05630f92550b,-7.677174,110.45501,1642378184
1,061fd420-fcd0-4923-8a2b-bbeee7c74872,-7.70582,110.31897,1642378196
2,061fd420-fcd0-4923-8a2b-bbeee7c74872,-7.705812,110.31898,1642378196
3,061fd420-fcd0-4923-8a2b-bbeee7c74872,-7.705812,110.31898,1642378220
4,b23130b2-1bdb-4f1d-b8b2-81d3204523c8,-7.84391,110.43813,1642378237


### Data reverse geocoding 

In [2]:
path2 = './DataGPS/RGmaret2022.csv'
df2 = pd.read_csv(path2)
df2 = df2.drop({'Unnamed: 0'},axis=1)
df2.head()

,maid,latitude,longitude,datetime_wib,geometry,Kelurahan/Desa,Kecamatan,Kabupaten
0,a3df8c97-4fa3-4b7d-9a05-f29ecdfb9b63,-7.7,110.400002,2022-03-01 00:00:00,POINT (110.4000015258789 -7.7),Harjobinangun,Pakem,Sleman
1,cab00b11-799a-4e09-8d8d-d6e4fdee0ee3,-7.7,110.400002,2022-03-01 00:00:17,POINT (110.4000015258789 -7.7),Harjobinangun,Pakem,Sleman
2,468dd682-ba29-4cf9-8eaf-622afd6ed11e,-7.7,110.400002,2022-03-01 00:00:24,POINT (110.4000015258789 -7.7),Harjobinangun,Pakem,Sleman
3,7d71b571-f70d-44e6-ab87-71c61a36f444,-7.7,110.400002,2022-03-01 00:00:30,POINT (110.4000015258789 -7.7),Harjobinangun,Pakem,Sleman
4,0b02589d-ab60-4831-b378-863ae2337d7a,-7.7,110.400002,2022-03-01 00:00:33,POINT (110.4000015258789 -7.7),Harjobinangun,Pakem,Sleman


In [ ]:
# df2.iloc[df2['maid'] == '8f60c037-8ff2-4ccd-bd6d-05630f92550b', [0,1,2,3]]
# df.iloc[df['maid'] == '8f60c037-8ff2-4ccd-bd6d-05630f92550b']

## Mulai Preprocess

## 1. Filter Unnecessary Area
Membatasi area data point pada range yg dilakukan analisis

In [4]:
def filter_area(gps, road, filename):
    gps_path = gps
    road_path = road
    
    tdf = skmob.TrajDataFrame.from_file(gps_path)
    area_shape = gpd.read_file(road_path)
    
    gdf_gps = GeoDataFrame(tdf, geometry=gpd.points_from_xy(tdf['longitude'], tdf['latitude']))
    filtered_data = gdf_gps[gdf_gps.geometry.within(area_shape.geometry.iloc[0])].copy()
    
    filtered_data['datetime_wib'] = pd.to_datetime(filtered_data['datetime_wib'])
    filtered_data['tanggal'] = filtered_data['datetime_wib'].dt.date
    
    filtered_data.to_csv(filename + '.csv', index=False)    

    filtered_data.info()
#     print(filtered_data.head())
    print("Filtering area for ", filename, "succeed")

In [5]:
gps = './DataGPS/RGoktober2021.csv'
road = './JaringanJalan/AreaTerban/clipping_boundary.geojson'
filename ='filter1_terban_okt'
filter_area(gps, road, filename)

<class 'geopandas.geodataframe.GeoDataFrame'>
Int64Index: 49830 entries, 2668973 to 10863180
Data columns (total 10 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   Unnamed: 0      49830 non-null  int64         
 1   maid            49830 non-null  object        
 2   latitude        49830 non-null  float64       
 3   longitude       49830 non-null  float64       
 4   datetime_wib    49830 non-null  datetime64[ns]
 5   geometry        49830 non-null  geometry      
 6   Kelurahan/Desa  49830 non-null  object        
 7   Kecamatan       49830 non-null  object        
 8   Kabupaten       49830 non-null  object        
 9   tanggal         49830 non-null  object        
dtypes: datetime64[ns](1), float64(2), geometry(1), int64(1), object(5)
memory usage: 4.2+ MB
Filtering area for  filter1_terban_okt succeed


In [ ]:
data = pd.read_csv(f'./{filename}.csv')
data.head()

,Unnamed: 0.1,Unnamed: 0,maid,latitude,longitude,datetime_wib,geometry,Kelurahan/Desa,Kecamatan,Kabupaten,tanggal
0,1488284,1488284,31cdeb51-76f4-46c2-98cf-a4454cb93e72,-7.793918,110.364883,2021-11-01 00:00:16,POINT (110.36488342285156 -7.793918),Kelurahan Sosromenduran,Gedongtengen,Kota Yogyakarta,2021-11-01
1,1488285,1488285,31cdeb51-76f4-46c2-98cf-a4454cb93e72,-7.793918,110.364883,2021-11-01 00:02:57,POINT (110.36488342285156 -7.793918),Kelurahan Sosromenduran,Gedongtengen,Kota Yogyakarta,2021-11-01
2,1488286,1488286,31cdeb51-76f4-46c2-98cf-a4454cb93e72,-7.793940,110.364883,2021-11-01 00:03:11,POINT (110.36488342285156 -7.79394),Kelurahan Sosromenduran,Gedongtengen,Kota Yogyakarta,2021-11-01
3,1488287,1488287,31cdeb51-76f4-46c2-98cf-a4454cb93e72,-7.793918,110.364883,2021-11-01 00:03:11,POINT (110.36488342285156 -7.793918),Kelurahan Sosromenduran,Gedongtengen,Kota Yogyakarta,2021-11-01
4,1488288,1488288,31cdeb51-76f4-46c2-98cf-a4454cb93e72,-7.793930,110.364868,2021-11-01 00:03:22,POINT (110.3648681640625 -7.79393),Kelurahan Sosromenduran,Gedongtengen,Kota Yogyakarta,2021-11-01


In [ ]:
data.nunique()

Unnamed: 0.1      597265
Unnamed: 0        597265
maid               13106
latitude            9314
longitude            837
datetime_wib      103084
geometry           58483
Kelurahan/Desa         5
Kecamatan              4
Kabupaten              1
tanggal               30
dtype: int64